### Imports

In [77]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from astropy.table import Table, vstack

### Data

In [78]:
mangaHIall = Table.read('input/mangaHIall_dnn.txt', format = 'ascii.commented_header')
kias7 = Table.read( 'input/kias1033_5_P-MJD-F_MPAJHU_ZdustOS_stellarMass_BPT_SFR_NSA_vflag.txt', format='ascii.commented_header')
voidCmax = Table.read('input/ALL_comoving_maximal.txt', format = 'ascii.commented_header')
voidC = Table.read('input/ALL_comoving_holes.txt', format = 'ascii.commented_header')

##  Calculating the distance from the HI galaxies to their nearest void edges

1) For void HI galaxies, we only want to find the distance to the nearest wall of the HI void.\
2) For wall HI galaxies, we first want to find the flag numbers that give us the shortest distance. Since, the flag numbers are not unique, we then have to find the distance from the HI wall to every single void center corresponding to that flag number in the voidC Table and at the end only store the smallest distance.

##### For HI voids

In [79]:
kias7_walls_boolean = kias7['vflag'] == 0.0
kias7_walls = kias7[kias7_walls_boolean]

In [80]:
manga_voids_boolean = mangaHIall['vflag'] == 1.0
voids_data = mangaHIall[manga_voids_boolean]

In [81]:
# Generates x_tree, y_tree, z_tree array for kias7_walls
x_tree = np.asarray(kias7_walls['x'])
x_tree = x_tree.astype(np.float64)
x_tree = x_tree.reshape(x_tree.shape[0],1)
y_tree = np.asarray(kias7_walls['y'])
y_tree = y_tree.astype(np.float64)
y_tree = y_tree.reshape(y_tree.shape[0],1)
z_tree = np.asarray(kias7_walls['z'])
z_tree = z_tree.astype(np.float64)
z_tree = z_tree.reshape(z_tree.shape[0],1)
xyz_tree_kias_walls = np.concatenate((x_tree,y_tree,z_tree), axis=1)

# Generates x_query, y_query, z_query array for manga
x_query = np.asarray(voids_data['x'])
x_query = x_query.astype(np.float64)
x_query = x_query.reshape(x_query.shape[0],1)
y_query = np.asarray(voids_data['y'])
y_query = y_query.astype(np.float64)
y_query = y_query.reshape(y_query.shape[0],1)
z_query = np.asarray(voids_data['z'])
z_query = z_query.astype(np.float64)
z_query = z_query.reshape(z_query.shape[0],1)
xyz_query_voids = np.concatenate((x_query,y_query,z_query), axis=1)


In [82]:
# Generate KDTree for kias7
tree_kias_walls = KDTree(xyz_tree_kias_walls)

In [83]:
nearest_distance_to_void_edge_voids, _ = tree_kias_walls.query(xyz_query_voids, k=1)

In [84]:
# mangaHIall['d_nve'] = np.NaN*np.ones(len(mangaHIall), dtype=float)
# mangaHIall['d_nve'][manga_voids_boolean] = nearest_distance_to_void_edge_voids
voids_data['d_nve'] = nearest_distance_to_void_edge_voids

##### For HI walls

In [85]:
manga_walls_boolean = mangaHIall['vflag'] == 0.0
walls_data = mangaHIall[manga_walls_boolean]

In [86]:
# Generates x_tree, y_tree, z_tree array for voidC
x_tree = np.asarray(voidC['x'])
x_tree = x_tree.astype(np.float64)
x_tree = x_tree.reshape(x_tree.shape[0],1)
y_tree = np.asarray(voidC['y'])
y_tree = y_tree.astype(np.float64)
y_tree = y_tree.reshape(y_tree.shape[0],1)
z_tree = np.asarray(voidC['z'])
z_tree = z_tree.astype(np.float64)
z_tree = z_tree.reshape(z_tree.shape[0],1)
xyz_tree_voidC = np.concatenate((x_tree,y_tree,z_tree), axis=1)

# Generates x_query, y_query, z_query array for manga
x_query = np.asarray(walls_data['x'])
x_query = x_query.astype(np.float64)
x_query = x_query.reshape(x_query.shape[0],1)
y_query = np.asarray(walls_data['y'])
y_query = y_query.astype(np.float64)
y_query = y_query.reshape(y_query.shape[0],1)
z_query = np.asarray(walls_data['z'])
z_query = z_query.astype(np.float64)
z_query = z_query.reshape(z_query.shape[0],1)
xyz_query_walls = np.concatenate((x_query,y_query,z_query), axis=1)

In [87]:
# Generate KDTree for voidC
tree_voidC = KDTree(xyz_tree_voidC)

In [88]:
nearest_distance_to_void_edge_walls, ind = tree_voidC.query(xyz_query_walls, k=1)

In [89]:
walls_data['flag'] = voidC['flag'][ind]

In [90]:
walls_data['d_nve'] = np.ones(len(walls_data), dtype = float)

x1= walls_data['x']
x2= voidC['x']

y1=walls_data['y']
y2=voidC['y']

z1=walls_data['z']
z2=voidC['z']

for i in range(len(walls_data)):
    dist_ini = 1000
    for j in range(len(voidC)):
        if walls_data['flag'][i] == voidC['flag'][j]:
            dist = abs(np.sqrt((x1[i]-x2[j])**2 + (y1[i]-y2[j])**2 + (z1[i]-z2[j])**2) - voidC['radius'][j])
            if dist < dist_ini:
                dist_ini = dist    
    walls_data['d_nve'][i] = dist        

##### Joining the voids and walls data and putting it into the mangaHIall file

In [91]:
mangaHIall = vstack([walls_data, voids_data])

##### Writing new mangaHIall file

In [92]:
mangaHIall.write('input/mangaHIall_nearestDistances.txt', format = 'ascii.commented_header', overwrite = True)